In [9]:
import pandas as pd
import numpy as np
import re
import json
from dateutil.parser import parse

OUT_FILE_PATH = 'menu_items.json'
MENU_PRIOR_UPDATED = 'master_menu/menu_backup.csv'
MENU_LATEST = 'master_menu/menu.csv'

SCHOOL_ID = 10
MLTYPE_CONST1 = ['Breakfast', 'Lunch', 'Dinner']

def clean_table(dfc):
    for key in dfc.keys():
        if dfc[key].isna().all() == True:
            dfc.drop(key, axis=1, inplace=True)
    return dfc

def meal(z):
    if "Breakfast" in z:
        return "7:30 am", "breakfast"
    elif "Lunch" in z:
        if "FLAME" in z:
            return "2:30 pm", "afterlunch"
        elif '500 DEGREES' in z:
            return "2:30 pm", "afterlunch"
        elif 'CARVED AND CRAFTED' in z:
            return "2:30 pm", "afterlunch"
        else:
            return "11:00 am", "lunch"
    elif "Dinner" in z:
        return "4:30 pm", "dinner"


def weekly_menu_dict(dfm, dfd, row_begin, row_end):
    if row_begin == 0:
        dfmw = dfm.loc[row_begin:row_end - 1, :].reset_index(drop=True)
        week_num = int(str(list(dfmw.keys())[-1]).strip('Week'))
        dfmw_keys = dfm.iloc[0]
        dfmw.drop(0, inplace=True)
    else:
        dfmw = dfm.loc[row_begin:row_end - 1, :]
        week_num = int(dfmw.loc[row_begin, list(dfmw.keys())[-1]].strip('Week'))
        dfmw_keys = dfm.iloc[row_begin + 1]
        dfmw.drop([row_begin, row_begin + 1], inplace=True)

    dfmw.columns = dfmw_keys
    dfmw.drop(dfmw_keys[-1], axis=1, inplace=True)
    dfmw = dfmw.reset_index(drop=True)

    endl = []
    for i in range(len(dfmw)):
        if 'end' in list(dfmw.iloc[i]):
            endl.append(i)
    dfmw.drop(endl, axis=0, inplace=True)
    dfmw = dfmw.reset_index(drop=True)

    first_key = list(dfmw_keys)[0]
    meal_ind = [list(dfmw.loc[dfmw[first_key].str.contains(ml, na=False)].index)
                for ml in MLTYPE_CONST1]
    meal_ind = sorted([t for tt in meal_ind for t in tt])

    MEAL_TYPES = []
    for i in range(len(meal_ind)):
        mt = dfmw.loc[meal_ind[i], first_key]
        if i < len(meal_ind) - 1:
            [MEAL_TYPES.append(mt) for ii in range(meal_ind[i + 1] - meal_ind[i])]
        else:
            [MEAL_TYPES.append(mt) for ii in range(len(dfmw) - 1 - meal_ind[i])]
            MEAL_TYPES.append(mt)
    dfmw['Meal_Types'] = MEAL_TYPES
    dfmw['Meal_Time'], dfmw['Meal'] = zip(*dfmw['Meal_Types'].apply(lambda z: meal(z)))

    DAYS = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    days_d = dict(zip(DAYS, list(dfmw.keys())))
    
    meals, timestamps, groups, names, weeknums, meals_recipe_num = [[] for i in range(6)]
    days_d['Week'] = week_num

    for d in DAYS:
        for meal_key, df_t in dfmw.groupby('Meal'):
            f = []
            df_t = df_t.reset_index(drop=True)
            s = re.sub(r"[\([{})\]]", '', days_d[d].strip('d')) + ' ' + df_t.loc[0, 'Meal_Time']
            tt = parse(s)
            timestamps.append(tt.strftime("%Y-%m-%d %H:%M:%SZ"))
            groups.append(meal_key)
            names.append(meal_key.title() + " on " + d + ', ' + tt.strftime("%B %d, %Y"))
            weeknums.append(week_num)
            for index, row in df_t.iterrows():
                try:
                    dish = float(row[days_d[d]])
                    f.append(row[days_d[d]])
                except:
                    pass
            f = [x for x in f if str(x) != 'nan']
            q = [int(dfd.loc[dfd['cafeteria_id'] == x, 'pk'].values[0])
                 for x in f if not dfd.loc[dfd['cafeteria_id'] == x, 'pk'].empty]
            meals.append(q)
            meals_recipe_num.append(f)
   
    MENU_WEEKLY = pd.DataFrame()
    MENU_WEEKLY['group'] = groups
    MENU_WEEKLY['timestamp'] = timestamps
    MENU_WEEKLY['name'] = names
    MENU_WEEKLY['items'] = meals
    MENU_WEEKLY['week_num'] = weeknums
    MENU_WEEKLY['items_recipe_num'] = meals_recipe_num

    MENU_WEEKLY['items'] = MENU_WEEKLY['items'].apply(lambda x: list(map(int, x)))
    MENU_WEEKLY.drop(MENU_WEEKLY[MENU_WEEKLY['items_recipe_num'].str.len() == 0].index, inplace=True)
    
    return MENU_WEEKLY, dfmw

def parse_fixture(d, out_filename):
    results = d.to_dict(orient='index')
    json_list=[]
    for r in range(len(results)):
        data = {'model': 'backend.MealSelection', 'pk': {}, 'fields': {}}
        f = results[r]
        data['pk'] = f['pk']
        for e in ['pk', 'week_num','items_recipe_num']:
            f.pop(e)
        f['school'] = SCHOOL_ID
        data['fields'] = f
        json_list.append(data)

    with open(out_filename, 'w') as out_file:
        json.dump(json_list, out_file, sort_keys=False)
    out_file.close()    


dfd = pd.read_csv(r'master_nutrition\nutrition_table.csv', converters={'cafeteria_id': lambda x: str(x)})
#dfm = pd.read_excel(r'menu\MenuWorks_W9-12_2022.xlsx', skiprows=11)
dfm = pd.read_excel(r'menu\MenuWorks_W11-13_2022.xlsx', skiprows=11)

dfm = clean_table(dfm)

k = str(list(dfm.keys())[0])
split_row = [0]
for i in range(len(dfm)):
    if 'TRIM' in str(dfm.loc[int(i), k]):
        split_row.append(i)

MENU = pd.DataFrame()
df_master = pd.DataFrame()
try:
    df_master = pd.read_csv(MENU_LATEST)
    df_master.to_csv(MENU_PRIOR_UPDATED,index=False) #save a backup of one version older
    df_master.drop(['pk'], axis=1, inplace=True)
    df_master['items'] = df_master['items'].apply(lambda x: eval(x))
except:
    pass

for mm in range(len(split_row)):
    if mm == (len(split_row)-1):
        menuw1, dfmw1 = weekly_menu_dict(dfm, dfd, split_row[mm], len(dfm))
    else:
        menuw1, dfmw1 = weekly_menu_dict(dfm, dfd, split_row[mm], split_row[mm + 1])
    MENU = pd.concat([MENU, menuw1]).reset_index(drop=True)

print(MENU[MENU['items_recipe_num'].str.len() != 0])



C:\Users\Penelope\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


         group             timestamp                                     name  \
0   afterlunch  2022-03-22 14:30:00Z    Afterlunch on Tuesday, March 22, 2022   
1    breakfast  2022-03-22 07:30:00Z     Breakfast on Tuesday, March 22, 2022   
2       dinner  2022-03-22 16:30:00Z        Dinner on Tuesday, March 22, 2022   
3        lunch  2022-03-22 11:00:00Z         Lunch on Tuesday, March 22, 2022   
4   afterlunch  2022-03-23 14:30:00Z  Afterlunch on Wednesday, March 23, 2022   
5    breakfast  2022-03-23 07:30:00Z   Breakfast on Wednesday, March 23, 2022   
6       dinner  2022-03-23 16:30:00Z      Dinner on Wednesday, March 23, 2022   
7        lunch  2022-03-23 11:00:00Z       Lunch on Wednesday, March 23, 2022   
8   afterlunch  2022-03-24 14:30:00Z   Afterlunch on Thursday, March 24, 2022   
9    breakfast  2022-03-24 07:30:00Z    Breakfast on Thursday, March 24, 2022   
10      dinner  2022-03-24 16:30:00Z       Dinner on Thursday, March 24, 2022   
11       lunch  2022-03-24 1

In [10]:
MENU.head(10)

,group,timestamp,name,items,week_num,items_recipe_num
0,afterlunch,2022-03-22 14:30:00Z,"Afterlunch on Tuesday, March 22, 2022",[],11,"[47527.3, 138463, 26162, 102407, 44726.1, 8225..."
1,breakfast,2022-03-22 07:30:00Z,"Breakfast on Tuesday, March 22, 2022","[1030, 1037, 1034, 1011, 1000]",11,"[2032.23, 23378.4, 105032, 616, 5828, 1178.9, ..."
2,dinner,2022-03-22 16:30:00Z,"Dinner on Tuesday, March 22, 2022","[1396, 1410, 1406, 1389, 1028, 1414, 1243, 130...",11,"[19368.53, 1120.3, 8965, 138462, 31169.3, 1033..."
3,lunch,2022-03-22 11:00:00Z,"Lunch on Tuesday, March 22, 2022","[1486, 1238, 1472, 1474, 1500, 1514, 1107, 144...",11,"[138913, 3978, 49762.1, 108470, 36567, 47204, ..."
4,afterlunch,2022-03-23 14:30:00Z,"Afterlunch on Wednesday, March 23, 2022","[1437, 1436, 1434]",11,"[114398, 8414.39, 5294.10, 102407, 44726.1, 82..."
5,breakfast,2022-03-23 07:30:00Z,"Breakfast on Wednesday, March 23, 2022","[1030, 1012, 1019, 1245, 1002]",11,"[2032.23, 18451.22, 22593.7, 608, 568.7, 10373..."
6,dinner,2022-03-23 16:30:00Z,"Dinner on Wednesday, March 23, 2022","[1393, 1412, 1397, 1374, 1405, 1373, 1415, 138...",11,"[16856.71, 32393.1, 8248, 102295, 102288, 4746..."
7,lunch,2022-03-23 11:00:00Z,"Lunch on Wednesday, March 23, 2022","[1465, 1478, 1477, 1483, 1479, 1502, 1498, 146...",11,"[45381.1, 3587, 97644, 14455, 518.1, 41001.10,..."
8,afterlunch,2022-03-24 14:30:00Z,"Afterlunch on Thursday, March 24, 2022",[1202],11,"[37320, 138954, 27213, 102407, 44726.1, 82258...."
9,breakfast,2022-03-24 07:30:00Z,"Breakfast on Thursday, March 24, 2022","[1030, 1314, 1027, 1032, 1002]",11,"[2032.23, 129218.7, 101631, 5303, 106424, 1033..."
